In [2]:
import numpy as np
from scipy.linalg import expm, sinm, cosm
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import math
from scipy.special import iv
from statsmodels.tsa.arima.model import ARIMA
import altair

In [200]:
data = pd.read_excel('C:/Users/michelj8/Documents/GitHub/METS/fred_md.xlsx')
data = data.set_index('sasdate')
series_list = data.columns
data.head()

,RPI,W875RX1,DPCERA3M086SBEA,CMRMTSPLx,RETAILx,INDPRO,IPFPNSS,IPFINAL,IPCONGD,IPDCONGD,...,DSERRG3M086SBEA,CES0600000008,CES2000000008,CES3000000008,UMCSENTx,MZMSL,DTCOLNVHFNM,DTCTHFNM,INVEST,VXOCLSx
sasdate,,,,,,,,,,,,,,,,,,,,,
Transform:,5.000,5.0,5.000,5.0000,5.00000,5.0000,5.0000,5.0000,5.0000,5.0000,...,6.000,6.00,6.00,6.00,2.0,6.0,6.0,6.0,6.0000,1.0
1959-01-01 00:00:00,2437.296,2288.8,17.302,292258.8329,18235.77392,22.6250,23.4581,22.1904,32.4078,21.9882,...,11.358,2.13,2.45,2.04,NaN,274.9,6476.0,12298.0,84.2043,NaN
1959-02-01 00:00:00,2446.902,2297.0,17.482,294429.5453,18369.56308,23.0681,23.7747,22.3827,32.6455,22.1036,...,11.375,2.14,2.46,2.05,NaN,276.0,6476.0,12298.0,83.5280,NaN
1959-03-01 00:00:00,2462.689,2314.0,17.647,293425.3813,18523.05762,23.4004,23.9186,22.4925,32.6455,22.5365,...,11.395,2.15,2.45,2.07,NaN,277.4,6508.0,12349.0,81.6405,NaN
1959-04-01 00:00:00,2478.744,2330.3,17.584,299331.6505,18534.46600,23.8989,24.2641,22.8221,33.1606,22.6807,...,11.436,2.16,2.47,2.08,NaN,278.1,6620.0,12484.0,81.8099,NaN


In [100]:
lag_length = 12
L_approx = np.zeros([lag_length,lag_length])

for row in range(1,lag_length):
    L_approx[row,row-1] =1

def stack_ts(ts):
    stacked_ts = np.zeros([lag_length,len(ts)-lag_length])
    
    for time in range(0, len(ts)-lag_length):
        stacked_ts[:,time] = ts[time:time+lag_length]
        
    return stacked_ts

def ols_mets(ts, grid_size):
    stacked_ts = stack_ts(ts)
    
    min_rho =  0.0
    filter_matrix = expm(min_rho*L_approx)
    resid = np.dot(filter_matrix,stacked_ts)[-1]
    min_alpha =  np.mean(resid)
    min_obj = np.square(np.std(resid))
    
    for rho in np.linspace(-2,2,grid_size):
        filter_matrix = expm(rho*L_approx)
        resid = np.dot(filter_matrix,stacked_ts)[-1]
        alpha =  np.mean(resid)
        obj = np.square(np.std(resid))
        
        if obj < min_obj:
            min_alpha = alpha
            min_rho = rho
            min_obj = obj

    return [min_alpha, min_rho, min_obj]    

In [238]:
def iterate_forecast(ts,alpha, rho, h_steps):
    stacked_ts = stack_ts(ts)
    filter_matrix = expm(rho*L_approx) 
    
    for h in range(0, h_steps):
        update = np.dot(filter_matrix, np.vstack((stacked_ts[1:], np.zeros(len(stacked_ts[0])))) )
        forecasts = alpha- update[-1]
        stacked_ts = np.vstack((stacked_ts[:-1], forecasts))
        
    return stacked_ts

In [239]:
ts = np.log(data['RPI'].iloc[2:]).diff().dropna().values

In [240]:
ols_mets(ts,100)

[0.0035225128179540745, 0.262626262626263, 5.0144662310894665e-05]

In [258]:
pd.DataFrame(iterate_forecast(ts,0, -.2, 0))

,0,1,2,3,4,5,6,7,8,9,...,715,716,717,718,719,720,721,722,723,724
0,0.006431,0.006498,0.005826,0.003108,-0.000586,-0.005695,0.000788,0.001223,0.007596,0.010197,...,0.001018,0.000837,0.009426,0.003021,0.002426,0.000802,-0.001509,-0.000194,0.000732,-0.000585
1,0.006498,0.005826,0.003108,-0.000586,-0.005695,0.000788,0.001223,0.007596,0.010197,0.003193,...,0.000837,0.009426,0.003021,0.002426,0.000802,-0.001509,-0.000194,0.000732,-0.000585,0.003939
2,0.005826,0.003108,-0.000586,-0.005695,0.000788,0.001223,0.007596,0.010197,0.003193,0.001145,...,0.009426,0.003021,0.002426,0.000802,-0.001509,-0.000194,0.000732,-0.000585,0.003939,0.001060
3,0.003108,-0.000586,-0.005695,0.000788,0.001223,0.007596,0.010197,0.003193,0.001145,0.001910,...,0.003021,0.002426,0.000802,-0.001509,-0.000194,0.000732,-0.000585,0.003939,0.001060,0.000689
4,-0.000586,-0.005695,0.000788,0.001223,0.007596,0.010197,0.003193,0.001145,0.001910,0.003426,...,0.002426,0.000802,-0.001509,-0.000194,0.000732,-0.000585,0.003939,0.001060,0.000689,0.004293
5,-0.005695,0.000788,0.001223,0.007596,0.010197,0.003193,0.001145,0.001910,0.003426,0.002403,...,0.000802,-0.001509,-0.000194,0.000732,-0.000585,0.003939,0.001060,0.000689,0.004293,-0.001965
6,0.000788,0.001223,0.007596,0.010197,0.003193,0.001145,0.001910,0.003426,0.002403,0.000789,...,-0.001509,-0.000194,0.000732,-0.000585,0.003939,0.001060,0.000689,0.004293,-0.001965,0.007643
7,0.001223,0.007596,0.010197,0.003193,0.001145,0.001910,0.003426,0.002403,0.000789,0.001832,...,-0.000194,0.000732,-0.000585,0.003939,0.001060,0.000689,0.004293,-0.001965,0.007643,0.006632
8,0.007596,0.010197,0.003193,0.001145,0.001910,0.003426,0.002403,0.000789,0.001832,-0.001624,...,0.000732,-0.000585,0.003939,0.001060,0.000689,0.004293,-0.001965,0.007643,0.006632,-0.015946
9,0.010197,0.003193,0.001145,0.001910,0.003426,0.002403,0.000789,0.001832,-0.001624,0.001274,...,-0.000585,0.003939,0.001060,0.000689,0.004293,-0.001965,0.007643,0.006632,-0.015946,0.120720


In [261]:
pd.DataFrame(stack_ts(ts))

,0,1,2,3,4,5,6,7,8,9,...,715,716,717,718,719,720,721,722,723,724
0,0.006431,0.006498,0.005826,0.003108,-0.000586,-0.005695,0.000788,0.001223,0.007596,0.010197,...,0.001018,0.000837,0.009426,0.003021,0.002426,0.000802,-0.001509,-0.000194,0.000732,-0.000585
1,0.006498,0.005826,0.003108,-0.000586,-0.005695,0.000788,0.001223,0.007596,0.010197,0.003193,...,0.000837,0.009426,0.003021,0.002426,0.000802,-0.001509,-0.000194,0.000732,-0.000585,0.003939
2,0.005826,0.003108,-0.000586,-0.005695,0.000788,0.001223,0.007596,0.010197,0.003193,0.001145,...,0.009426,0.003021,0.002426,0.000802,-0.001509,-0.000194,0.000732,-0.000585,0.003939,0.001060
3,0.003108,-0.000586,-0.005695,0.000788,0.001223,0.007596,0.010197,0.003193,0.001145,0.001910,...,0.003021,0.002426,0.000802,-0.001509,-0.000194,0.000732,-0.000585,0.003939,0.001060,0.000689
4,-0.000586,-0.005695,0.000788,0.001223,0.007596,0.010197,0.003193,0.001145,0.001910,0.003426,...,0.002426,0.000802,-0.001509,-0.000194,0.000732,-0.000585,0.003939,0.001060,0.000689,0.004293
5,-0.005695,0.000788,0.001223,0.007596,0.010197,0.003193,0.001145,0.001910,0.003426,0.002403,...,0.000802,-0.001509,-0.000194,0.000732,-0.000585,0.003939,0.001060,0.000689,0.004293,-0.001965
6,0.000788,0.001223,0.007596,0.010197,0.003193,0.001145,0.001910,0.003426,0.002403,0.000789,...,-0.001509,-0.000194,0.000732,-0.000585,0.003939,0.001060,0.000689,0.004293,-0.001965,0.007643
7,0.001223,0.007596,0.010197,0.003193,0.001145,0.001910,0.003426,0.002403,0.000789,0.001832,...,-0.000194,0.000732,-0.000585,0.003939,0.001060,0.000689,0.004293,-0.001965,0.007643,0.006632
8,0.007596,0.010197,0.003193,0.001145,0.001910,0.003426,0.002403,0.000789,0.001832,-0.001624,...,0.000732,-0.000585,0.003939,0.001060,0.000689,0.004293,-0.001965,0.007643,0.006632,-0.015946
9,0.010197,0.003193,0.001145,0.001910,0.003426,0.002403,0.000789,0.001832,-0.001624,0.001274,...,-0.000585,0.003939,0.001060,0.000689,0.004293,-0.001965,0.007643,0.006632,-0.015946,0.120720
